In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
from bs4 import BeautifulSoup
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
def get_voting_results(section_link, partial_link_text):
    # section_link should lead to the page where votes from year xxx to year yyy are located 
    #(example: https://www.bk.admin.ch/ch/d/pore/va/vab_2_2_4_1_2011_2020.html)
    
    driver = webdriver.Chrome()
    driver.get(section_link)
    alle_abstimmungen = driver.find_elements_by_partial_link_text(partial_link_text)
    data = pd.DataFrame()
    
    for n_abstimmung in range(len(alle_abstimmungen)):
        driver.find_elements_by_partial_link_text(partial_link_text)[n_abstimmung].click()
        
        url_1 = driver.current_url
        for v in range(len(driver.find_elements_by_link_text("Kantonsresultate"))):
                
            ActionChains(driver).move_to_element(driver.find_elements_by_link_text("Kantonsresultate")[v]).perform()
            driver.find_elements_by_link_text("Kantonsresultate")[v].click()
            element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, 
                                                                                        "//*[@id='content']/div[3]/article/h3")))
            aa = driver.find_element_by_xpath("//*[@id='content']/div[3]/article/h3")
            bb = driver.find_element_by_xpath("//*[@id='contentNavigation']/a")

            try:
                date_ini = re.search(r'\d{2}.\d{2}.\d{4}', aa.text).group()
            except AttributeError:
                date_ini = np.nan

            date_abs = re.search(r'\d{2}.\d{2}.\d{4}', bb.text).group() # date of voting

            try:
                topic = re.findall(r"\'(.+?)\'", aa.text)[0]
            except IndexError:
                topic = aa.text
            url = driver.current_url

            html = driver.page_source
            soup = BeautifulSoup(html,'html.parser')
            div = soup.select_one("table")
            table=pd.read_html(str(div))

            if len(data) == 0:
                data = table[0]
                data["Date_ini"] = [date_ini]*len(data)
                data["Date_abs"] = [date_abs]*len(data)
                data["Topic"] = [topic]*len(data)
                data["Link"] = [url]*len(data)
            else:
                data1 = table[0]
                data1["Date_ini"] = [date_ini]*len(data1)
                data1["Date_abs"] = [date_abs]*len(data1)
                data1["Topic"] = [topic]*len(data1)
                data1["Link"] = [url]*len(data1)

                data = pd.concat([data, data1])
                
            driver.get(url_1)
            
        driver.get(section_link)
    
    return data

In [4]:
for years in ["2011_2020", "2001_2010", "1991_1999", "2000", "1981_1990", "1971_1980", "1961_1970", "1951_1960"]:
    st = "." + years[:2]
    link = "https://www.bk.admin.ch/ch/d/pore/va/vab_2_2_4_1_" + years + ".html"
    globals()["votings_" + years] = get_voting_results(link, st)

In [193]:
votings = pd.concat([votings_1951_1960, votings_1961_1970, votings_1971_1980, votings_1981_1990, votings_2000, 
                     votings_1991_1999, votings_2001_2010, votings_2011_2020])

In [197]:
votings[:2]

,Kanton,Stimmber.,Einger.,% Stimmbet.,Ja,Nein,% Ja,% Nein,Date_ini,Date_abs,Topic,Link,Stände Ja,Stände Nein
0,Zürich,239'976,164'984,68.75%,65'251,94'948,40.7%,59.3%,NaN,25.02.1951,Bundesbeschluss über den Transport von Persone...,https://www.bk.admin.ch/ch/d/pore/va/19510225/...,NaN,NaN
1,Bern,247'324,93'313,37.73%,44'993,47'567,48.6%,51.4%,NaN,25.02.1951,Bundesbeschluss über den Transport von Persone...,https://www.bk.admin.ch/ch/d/pore/va/19510225/...,NaN,NaN


In [5]:
m_inits = pd.read_excel("military_links.xlsx", header = None)

In [6]:
def init_text(link, driverr):
    driverr.get(link)
    driverr.find_elements_by_link_text("Übersicht")[0].click()
    try:
        driverr.find_elements_by_link_text("Detailangaben zur Volksinitiative")[0].click()
        driverr.find_elements_by_link_text("Die Initiative im Wortlaut")[0].click()
        cc = driverr.find_element_by_xpath("//*[@id='content']/div[3]/article")
        ini_t = cc.text.replace("\n", "")
    except IndexError:
        ini_t = np.nan

    return ini_t

In [7]:
driver3r = webdriver.Chrome()

In [8]:
descs = []
for l in m_inits[0]:
    ff = init_text(l, driver3r)
    descs.append(ff)